# Excercise 02 - Training an ML model

Welcome to the second hands-on excercise to get started with CyclOps!

We will use the dataset introduced in the first excercise to train an ML model! At the end of this excercise, you will be able to:

1. Create training and validation datasets using the [🤗 Datasets](https://github.com/huggingface/datasets) library
2. Train a baseline ML model using CyclOps

## Step 01 - Install CyclOps

CyclOps is available as a [python package](https://pypi.org/project/pycyclops/) and can be installed using ``pip``. Note that we now install ``CyclOps`` with and extra dependency ``xgboost`` since we will be using the [xgboost](https://xgboost.readthedocs.io/en/stable/python/python_api.html) library.

``Colab`` would ask you to restart the session, which is normal. Click on ``Restart Session`` and re-run the cell to install ``CyclOps``.

**NOTE**: We uninstall ``cupy`` from the colab runtime to avoid conflicts with ``CyclOps`` which would attempt to use ``cupy`` if it is installed. Since the runtime does not support GPUs, we will uninstall ``cupy``.

In [ ]:
!pip uninstall cupy-cuda12x -y
!pip install 'pycyclops[xgboost]'
!pip install ucimlrepo

## Step 02 - Load and prepare the [Diabetes 130-US Hospitals for Years 1999-2008](https://archive.ics.uci.edu/dataset/296/diabetes+130-us+hospitals+for+years+1999-2008) dataset

We already explored the [Diabetes 130-US Hospitals for Years 1999-2008](https://archive.ics.uci.edu/dataset/296/diabetes+130-us+hospitals+for+years+1999-2008) dataset. Now, we will prepare select a subset from the dataset, and process it so that we can create training and validation datasets.

In [ ]:
from ucimlrepo import fetch_ucirepo

diabetes_130_data = fetch_ucirepo(
    id=296
)  # This ID specifically corresponds to the Diabetes 130 dataset
features = diabetes_130_data["data"]["features"]
targets = diabetes_130_data["data"]["targets"]
metadata = diabetes_130_data["metadata"]
variables = diabetes_130_data["variables"]

We will transform the readmitted outcome variable into binary 0/1 labels!

In [ ]:
def transform_label(value):
    """Transform string labels of readmission into 0/1 binary labels.

    Parameters
    ----------
    value: str
        Input value

    Returns
    -------
    int
        0 if not readmitted or if greater than 30 days, 1 if less than 30 days

    """
    if value in ["NO", ">30"]:
        return 0
    if value == "<30":
        return 1

    raise ValueError("Unexpected value for readmission!")


df = features
targets.loc[:, "readmitted"] = targets["readmitted"].apply(transform_label)
df.loc[:, "readmitted"] = targets["readmitted"]

Due to the large size of the dataset (around 100k examples), we will choose a small subset for training an ML model!

In [ ]:
df = df[0:1000]

In [ ]:
import pandas as pd
from datasets import Dataset
from datasets.features import ClassLabel
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

from cyclops.data.df.feature import TabularFeatures
from cyclops.models.catalog import create_model
from cyclops.tasks import BinaryTabularClassificationTask

In [ ]:
RANDOM_SEED = 85
NAN_THRESHOLD = 0.75
TRAIN_SIZE = 0.8

We previously looked at the missingness in the data. Let's remove features that are NaNs or have just a single unique value!

In [ ]:
features_to_remove = []
for col in df:
    if len(df[col].value_counts()) <= 1:
        features_to_remove.append(col)
df = df.drop(columns=features_to_remove)

It is also important that we understand the class imbalance and use it to train our binary classifier to weight the class with fewer examples accordingly.

In [ ]:
class_counts = df["readmitted"].value_counts()
class_ratio = class_counts[0] / class_counts[1]
print(class_ratio, class_counts)

From the features in the dataset, we select all of them to train the model!

In [ ]:
features_list = list(df.columns)
features_list.remove("readmitted")
features_list = sorted(features_list)

### Identifying feature types

Cyclops `TabularFeatures` class helps to identify feature types, an essential step before preprocessing the data. Understanding feature types (numerical/categorical/binary) allows us to apply appropriate preprocessing steps for each type.

In [ ]:
tab_features = TabularFeatures(
    data=df.reset_index(),
    features=features_list,
    by="index",
    targets="readmitted",
)
print(tab_features.types)

### Creating data preprocessors

We create a data preprocessor using sklearn's ColumnTransformer. This helps in applying different preprocessing steps to different columns in the dataframe. For instance, binary features might be processed differently from numeric features.

In [ ]:
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())],
)

binary_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="most_frequent"))],
)

In [ ]:
numeric_features = sorted((tab_features.features_by_type("numeric")))
numeric_indices = [
    df[features_list].columns.get_loc(column) for column in numeric_features
]
print(numeric_features)

In [ ]:
binary_features = sorted(tab_features.features_by_type("binary"))
binary_features.remove("readmitted")
ordinal_features = sorted(
    tab_features.features_by_type("ordinal")
    + ["medical_specialty", "diag_1", "diag_2", "diag_3"]
)
binary_indices = [
    df[features_list].columns.get_loc(column) for column in binary_features
]
ordinal_indices = [
    df[features_list].columns.get_loc(column) for column in ordinal_features
]
print(binary_features, ordinal_features)

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_indices),
        (
            "onehot",
            OneHotEncoder(handle_unknown="ignore", sparse_output=False),
            binary_indices + ordinal_indices,
        ),
    ],
    remainder="passthrough",
)

## Creating Hugging Face Dataset

We convert our processed Pandas dataframe into a Hugging Face dataset, a powerful and easy-to-use data format which is also compatible with CyclOps modules. The dataset is then split to train and test sets.

In [ ]:
dataset = Dataset.from_pandas(df)
dataset.cleanup_cache_files()
print(dataset)

In [ ]:
dataset = dataset.cast_column("readmitted", ClassLabel(num_classes=2))
dataset = dataset.train_test_split(
    train_size=TRAIN_SIZE,
    stratify_by_column="readmitted",
    seed=RANDOM_SEED,
)

## Model Creation

CyclOps model registry allows for straightforward creation and selection of models. This registry maintains a list of pre-configured models, which can be instantiated with a single line of code. Here we use a [XGBoost classifier](https://xgboost.readthedocs.io/en/stable/python/python_api.html) to fit a binary classification model. The model configurations can be passed to `create_model` based on the parameters for the ``XGBClassifier``.

In [ ]:
model_name = "xgb_classifier"
model = create_model(model_name, random_state=123)

## Task Creation

We use Cyclops tasks to define our model's task (in this case, readmission prediction), train the model, make predictions, and evaluate performance. Cyclops task classes encapsulate the entire ML pipeline into a single, cohesive structure, making the process smooth and easy to manage.

In [ ]:
readmission_prediction_task = BinaryTabularClassificationTask(
    {model_name: model},
    task_features=features_list,
    task_target="outcome",
)

In [ ]:
readmission_prediction_task.list_models()

## Training

If `best_model_params` is passed to the `train` method, the best model will be selected after the hyperparameter search. The parameters in `best_model_params` indicate the values to create the parameters grid.

Note that the data preprocessor needs to be passed to the tasks methods if the Hugging Face dataset is not already preprocessed. 

In [ ]:
best_model_params = {
    "n_estimators": [100, 250, 500],
    "learning_rate": [0.1, 0.01],
    "max_depth": [2, 5],
    "reg_lambda": [0, 1, 10],
    "colsample_bytree": [0.7, 0.8, 1],
    "gamma": [0, 1, 2, 10],
    "method": "random",
    "scale_pos_weight": [int(class_ratio)],
}
readmission_prediction_task.train(
    dataset["train"],
    model_name=model_name,
    transforms=preprocessor,
    best_model_params=best_model_params,
)

In [ ]:
model_params = readmission_prediction_task.list_models_params()[model_name]
print(model_params)

## Prediction

The prediction output can be either the whole Hugging Face dataset with the prediction columns added to it or the single column containing the predicted values.

In [ ]:
y_pred = readmission_prediction_task.predict(
    dataset["test"],
    model_name=model_name,
    transforms=preprocessor,
    proba=True,
    only_predictions=True,
)
prediction_df = pd.DataFrame(
    {
        "y_prob": [y_pred_i[1] for y_pred_i in y_pred],
        "y_true": dataset["test"]["outcome"],
        "gender": dataset["test"]["gender"],
    }
)